In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl


In [26]:
pd.read_csv('data/season-0910_csv.csv')[['Date', 'HomeTeam', 'AwayTeam','FTR', 'HTHG',
       'B365H', 'B365H','B365D','BbAvH', 'BbAvD', 'BbAvA']]


,Date,HomeTeam,AwayTeam,FTR,HTHG,B365H,B365H,B365D,BWA,BbAvH,BbAvD,BbAvA
0,2009-08-15,Aston Villa,Wigan,A,0,1.67,1.67,3.60,5.00,1.66,3.51,5.33
1,2009-08-15,Blackburn,Man City,A,0,3.60,3.60,3.25,2.00,3.37,3.24,2.12
2,2009-08-15,Bolton,Sunderland,A,0,2.25,2.25,3.25,3.15,2.24,3.20,3.15
3,2009-08-15,Chelsea,Hull,H,1,1.17,1.17,6.50,13.00,1.17,6.26,16.39
4,2009-08-15,Everton,Arsenal,A,0,3.20,3.20,3.25,2.25,3.07,3.21,2.28
...,...,...,...,...,...,...,...,...,...,...,...,...
375,2010-05-09,Everton,Portsmouth,H,0,1.29,1.29,5.50,10.00,1.28,5.23,10.38
376,2010-05-09,Hull,Liverpool,D,0,6.50,6.50,3.80,1.55,6.22,3.91,1.54
377,2010-05-09,Man United,Stoke,H,2,1.13,1.13,8.00,17.00,1.12,7.98,20.73
378,2010-05-09,West Ham,Man City,D,1,4.00,4.00,3.60,1.78,4.10,3.51,1.85


In [34]:
letter_to_result = {'H': 1, 'D': 0, 'A': -1}

def basic_clean(path, letter_to_result = {'H': 1, 'D': 0, 'A': -1}):
    """ (str) -> pandas.core.frame.DataFrame
    This function receives the path to raw csv file,
    afther that she reads it and performs main cleaning,
    and returns a single Dataframe
    """
    df = pd.read_csv(path)
    # Isolating useful betting columns, and getting rid of all others
    betting_columns = df[['B365H', 'B365D','B365A','BbAvH', 'BbAvD', 'BbAvA']]
    stats = df[df.columns[:23]]

    # As it turns out, Referee and div does not corellate with the result
    stats = (stats.drop('Div', axis=1)
         .drop('Referee', axis=1))
    # Merging stats and betting history:
    matches = pd.merge(stats, betting_columns, left_index=True, right_index=True)
    # Set all dates to the same format:
    matches['Date'] = pd.to_datetime(matches['Date'])
    matches['FTR'] = matches['FTR'].apply(lambda x: letter_to_result[x])
    return matches
basic_clean('data/season-0910_csv.csv', letter_to_result)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,HY,AY,HR,AR,B365H,B365D,B365A,BbAvH,BbAvD,BbAvA
0,2009-08-15,Aston Villa,Wigan,0,2,-1,0,1,A,11,...,2,2,0,0,1.67,3.60,5.50,1.66,3.51,5.33
1,2009-08-15,Blackburn,Man City,0,2,-1,0,1,A,17,...,2,1,0,0,3.60,3.25,2.10,3.37,3.24,2.12
2,2009-08-15,Bolton,Sunderland,0,1,-1,0,1,A,11,...,2,1,0,0,2.25,3.25,3.25,2.24,3.20,3.15
3,2009-08-15,Chelsea,Hull,2,1,1,1,1,D,26,...,1,2,0,0,1.17,6.50,21.00,1.17,6.26,16.39
4,2009-08-15,Everton,Arsenal,1,6,-1,0,3,A,8,...,0,0,0,0,3.20,3.25,2.30,3.07,3.21,2.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2010-05-09,Everton,Portsmouth,1,0,1,0,0,D,21,...,0,0,0,0,1.29,5.50,11.00,1.28,5.23,10.38
376,2010-05-09,Hull,Liverpool,0,0,0,0,0,D,11,...,1,0,0,0,6.50,3.80,1.57,6.22,3.91,1.54
377,2010-05-09,Man United,Stoke,4,0,1,2,0,H,18,...,2,0,0,0,1.13,8.00,23.00,1.12,7.98,20.73
378,2010-05-09,West Ham,Man City,1,1,0,1,1,D,12,...,2,0,0,0,4.00,3.60,1.91,4.10,3.51,1.85


In [35]:
def all_years():
    """ None -> pandas.core.frame.DataFrame
    Retreave all files from seasons 9/10 till season 18/19,
    clean them with basic_clean function and finally
    merge them together into one Dataframe
    """

    season_list = []
    # Iterating through all seasons and appending dataframes of them to list
    for i in range(9, 19):
        season_start = '09' if i == 9 else str(i) 
        path = f'data/season-{season_start}{str(i + 1)}_csv.csv'
        season_list.append(basic_clean(path))

    # Concat all the dataframes in the list together and return it
    seasons = (pd.concat(season_list)
               .reset_index()
               .drop('index', axis=1))
    seasons.to_csv(r'data/seasons_merged.csv')
    return seasons

all_years().head()
        

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,HY,AY,HR,AR,B365H,B365D,B365A,BbAvH,BbAvD,BbAvA
0,2009-08-15,Aston Villa,Wigan,0,2,-1,0,1,A,11,...,2,2,0,0,1.67,3.60,5.50,1.66,3.51,5.33
1,2009-08-15,Blackburn,Man City,0,2,-1,0,1,A,17,...,2,1,0,0,3.60,3.25,2.10,3.37,3.24,2.12
2,2009-08-15,Bolton,Sunderland,0,1,-1,0,1,A,11,...,2,1,0,0,2.25,3.25,3.25,2.24,3.20,3.15
3,2009-08-15,Chelsea,Hull,2,1,1,1,1,D,26,...,1,2,0,0,1.17,6.50,21.00,1.17,6.26,16.39
4,2009-08-15,Everton,Arsenal,1,6,-1,0,3,A,8,...,0,0,0,0,3.20,3.25,2.30,3.07,3.21,2.28
